# Begin

### Import Statements

In [1]:
import pandas as pd
import itertools
from scipy.stats import spearmanr

### Load DataFrames

In [2]:
min_kls = pd.read_csv('test-kl-figures/min_kls_new.csv', index_col=[0, 1, 2])
min_kls.columns.name = 'Algorithm'

In [3]:
y_normalized_kls = pd.read_csv('test-kl-figures/y_normalized_kls.csv', index_col=[0, 1, 2])

In [4]:
kl_at_scales_5_20 = pd.read_csv('test-kl-figures/kl_at_5_and_20.csv', index_col=[0, 1, 2])
kl_at_scales_1_10 = pd.read_csv('test-kl-figures/kl_at_1_and_10.csv', index_col=[0, 1, 2])
kl_at_scales = pd.concat([kl_at_scales_1_10, kl_at_scales_5_20], axis=0)
kl_at_scales.columns.name = 'Algorithm'
kl_at_scales = kl_at_scales.unstack().reorder_levels(["Scale", "Algorithm"], axis=1).sort_index(axis=1)

### Common Functions

In [5]:
def get_kl_stats(df):
    """
    Calculate statistics (min, max, mean, standard deviation, median) for each algorithm in `df`
    """
    kl_stats = df.agg([pd.Series.min, pd.Series.max, pd.Series.mean, pd.Series.std, pd.Series.median]).T

    kl_stats.index.rename('Statistic', level=1, inplace=True)

    colors = {
        "min": "background-color: #2F2D2E; color: white",
        "max": "background-color: #808080; color: white",
        "mean": "background-color: #536878; color: white",
        "std": "background-color: #493D31; color: white",
        "median": "background-color: #323F48: color: white",
    }


    def make_pretty(styler):
        styler.set_caption('Statistics')
        styler.apply(lambda row : [colors.get(row.name[1], "")] * len(row), axis=1)
        styler.map_index(lambda stat : colors.get(stat, ""), axis=0, level=1)
        styler.set_table_styles(
        [{'selector': 'td, th', 'props': [('border', '1px solid black')]}]
    )
        return styler

    # Apply the Styler
    return kl_stats.style.pipe(make_pretty)

In [6]:
def percentage_per_order(df: pd.DataFrame, drop_UMAP: bool):
    """
    Find the percentages of instances where the metric in `df` follows a certain order (e.g. MDS >= TSNE >= RANDOM)
    """
    results = {}

    if drop_UMAP:
        permutations = itertools.permutations(['TSNE', 'MDS', 'RANDOM'])
    else:
        permutations = itertools.permutations(['TSNE', 'UMAP', 'MDS', 'RANDOM'])


    for perm in permutations:
        condition = pd.Series(True, index=df.index)
        for i in range(1, len(perm)):
            condition &= df[perm[i-1]] <= df[perm[i]]

        percentage = (condition).mean() * 100
        # results[" < ".join(perm)] = f"{percentage:.3}%"
        results[" < ".join(perm)] = percentage


    return results

In [7]:
def ranks_for_spearman(df: pd.DataFrame, drop_UMAP: bool):
    """
    Gives each entry a rank depending on the order of least KL between the algorithms
    """
    if drop_UMAP:
        permutations = itertools.permutations(['TSNE', 'MDS', 'RANDOM'])
    else:
        permutations = reversed(list(itertools.permutations(df.columns)))

    ranked_df = pd.DataFrame(100, index=df.index, columns=["Rank"])

    for rank, perm in enumerate(permutations):
        condition = pd.Series(True, index=df.index)
        for i in range(1, len(perm)):
            condition &= df[perm[i-1]] <= df[perm[i]]

        ranked_df[condition] = rank

    return ranked_df
        

# Analysis: Min KLs

### Intro

In [8]:
min_kls.info()
min_kls.head(15)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 460 entries, ('auto-mpg', 'Run 0', 'x') to ('wine', 'Run 9', 'y')
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   RANDOM  460 non-null    float64
 1   MDS     460 non-null    float64
 2   UMAP    460 non-null    float64
 3   TSNE    460 non-null    float64
dtypes: float64(4)
memory usage: 17.1+ KB


Algorithm               RANDOM       MDS      UMAP      TSNE
Dataset  Run   Coord                                        
auto-mpg Run 0 x      0.334751  6.365166  0.518053  0.197916
               y      3.653443  2.659041  2.744190  2.559557
         Run 1 x      0.144634  6.404407  0.494673  0.200003
               y      3.654084  2.654081  2.746081  2.564123
         Run 2 x      0.000006  6.470418  0.549763  0.193610
               y      3.654109  2.649762  2.707762  2.580860
         Run 3 x      0.000006  6.434200  0.483282  0.192236
               y      3.654109  2.652285  2.774298  2.567548
         Run 4 x      0.000005  6.566125  0.475995  0.200524
               y      3.654109  2.639707  2.757787  2.540521
         Run 5 x      0.150067  6.494810  0.456903  0.203751
               y      3.654081  2.652813  2.687703  2.542320
         Run 6 x      0.099498  6.611997  0.540509  0.195360
               y      3.654103  2.637197  2.636219  2.585778
         Run 7 x      0.000004  6.457629  0.547171  0.195368

### Check within search range

* Since minimum-finding function (`scipy.optimize.minimize_scalar`) was bounded by (0, 300), should check if the minimum actually lies beyond 300 for any embedding

In [9]:
minimum_beyond_bound = ((min_kls.loc[:, :, 'x'] > 250)).any(axis=None)
print("There are some minima beyond 250: ", minimum_beyond_bound)

There are some minima beyond 250:  False


* For which datasets do the graphs need to be drawn for scales greater than 15?

In [10]:
thresh = 15
# set([idx[0] for idx in min_kls.loc[:, :, 'x'][min_kls.loc[:, :, 'x'] > thresh].dropna(how='all').index])
min_kls.loc[:, :, 'x'][min_kls.loc[:, :, 'x'] > thresh].dropna(how='all').style.background_gradient('viridis', vmin=thresh, vmax=250)

### KL Statistics of Each Algorithm

In [11]:
get_kl_stats(min_kls.groupby('Coord'))

# Analysis: Y-Normalized KLs

### Intro

In [12]:
y_normalized_kls.info()
y_normalized_kls.head(15)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 460 entries, ('auto-mpg', 'Run 0', 'x') to ('wine', 'Run 9', 'y')
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   RANDOM  460 non-null    float64
 1   MDS     460 non-null    float64
 2   UMAP    460 non-null    float64
 3   TSNE    460 non-null    float64
dtypes: float64(4)
memory usage: 17.1+ KB


RANDOM       MDS      UMAP      TSNE
Dataset  Run   Coord                                        
auto-mpg Run 0 x      0.736095  0.504811  0.037664  0.017680
               y      3.659560  3.496947  3.457386  3.381452
         Run 1 x      0.743537  0.516091  0.040224  0.018109
               y      3.664046  3.491325  3.413803  3.377061
         Run 2 x      0.731760  0.508052  0.051974  0.017953
               y      3.666392  3.495301  3.380314  3.371496
         Run 3 x      0.745632  0.506356  0.047378  0.016292
               y      3.665285  3.495957  3.375667  3.389427
         Run 4 x      0.749013  0.508962  0.048340  0.017269
               y      3.664596  3.494545  3.351805  3.385543
         Run 5 x      0.766160  0.504216  0.036613  0.017405
               y      3.664640  3.497259  3.374742  3.390907
         Run 6 x      0.728485  0.512183  0.043899  0.017730
               y      3.663697  3.493011  3.384443  3.404703
         Run 7 x      0.741550  0.508405  0.035247  0.016762

### KL Statistics of Each Algorithm

In [13]:
get_kl_stats(y_normalized_kls.groupby('Coord'))

# Analysis: KL at Scales (1 and 10)

### Intro

In [14]:
kl_at_scales.info()
kl_at_scales.head(15)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 230 entries, ('auto-mpg', 'Run 0') to ('wine', 'Run 9')
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   (1, MDS)      230 non-null    float64
 1   (1, RANDOM)   230 non-null    float64
 2   (1, TSNE)     230 non-null    float64
 3   (1, UMAP)     230 non-null    float64
 4   (5, MDS)      230 non-null    float64
 5   (5, RANDOM)   230 non-null    float64
 6   (5, TSNE)     230 non-null    float64
 7   (5, UMAP)     230 non-null    float64
 8   (10, MDS)     230 non-null    float64
 9   (10, RANDOM)  230 non-null    float64
 10  (10, TSNE)    230 non-null    float64
 11  (10, UMAP)    230 non-null    float64
 12  (20, MDS)     230 non-null    float64
 13  (20, RANDOM)  230 non-null    float64
 14  (20, TSNE)    230 non-null    float64
 15  (20, UMAP)    230 non-null    float64
dtypes: float64(16)
memory usage: 30.4+ KB


Scale                 1                                       5             \
Algorithm            MDS    RANDOM      TSNE      UMAP       MDS    RANDOM   
Dataset  Run                                                                 
auto-mpg Run 0  3.258463  3.672997  2.957726  2.810229  2.671279  4.161121   
         Run 1  3.258211  3.679646  2.959572  2.817809  2.666992  4.199261   
         Run 2  3.257867  3.683661  2.990390  2.763595  2.663695  4.196272   
         Run 3  3.257554  3.681004  2.972977  2.848668  2.665593  4.193227   
         Run 4  3.256776  3.679377  2.936701  2.831104  2.655259  4.181618   
         Run 5  3.258408  3.678597  2.932687  2.767105  2.666965  4.177248   
         Run 6  3.257174  3.679913  2.999546  2.695380  2.653555  4.193887   
         Run 7  3.258667  3.680985  2.942217  2.846319  2.668027  4.194356   
         Run 8  3.257886  3.681088  2.968367  2.795132  2.656456  4.194350   
         Run 9  3.257652  3.681807  2.968792  2.749498  2.660888  4.209832   
bank     Run 0  4.450905  5.930230  2.211930  3.869236  3.696474  6.440256   
         Run 1  4.384951  5.931105  2.253703  3.872874  3.608967  6.444942   
         Run 2  4.400867  5.933639  2.234368  3.850025  3.625367  6.447814   
         Run 3  4.452050  5.932951  2.204062  3.856962  3.727842  6.440768   
         Run 4  4.448659  5.935971  2.211096  3.860266  3.692299  6.453059   

Scale                                     10                                \
Algorithm           TSNE      UMAP       MDS    RANDOM      TSNE      UMAP   
Dataset  Run                                                                 
auto-mpg Run 0  3.593373  3.576499  2.698553  4.570605  3.848491  4.074403   
         Run 1  3.604868  3.606241  2.692613  4.619551  3.867803  4.129957   
         Run 2  3.655485  3.517261  2.686477  4.607569  3.945222  4.024686   
         Run 3  3.604901  3.629723  2.689819  4.609424  3.861281  4.150589   
         Run 4  3.586441  3.604224  2.673962  4.595642  3.850438  4.106868   
         Run 5  3.581731  3.542717  2.688451  4.583897  3.847234  4.052495   
         Run 6  3.674461  3.459896  2.670410  4.612540  3.952760  3.977362   
         Run 7  3.580406  3.579910  2.691199  4.614375  3.835239  4.094509   
         Run 8  3.603866  3.550324  2.674497  4.607555  3.861908  4.073102   
         Run 9  3.599729  3.522231  2.681895  4.646386  3.854367  4.042906   
bank     Run 0  2.688468  2.863362  3.629560  6.855971  3.709427  2.779101   
         Run 1  2.698208  2.847373  3.548116  6.860778  3.693202  2.747697   
         Run 2  2.733243  2.844154  3.559066  6.860975  3.760239  2.757384   
         Run 3  2.727328  2.844733  3.674662  6.852195  3.767335  2.754224   
         Run 4  2.690428  2.849515  3.625725  6.868300  3.710279  2.759673   

Scale                 20                                
Algorithm            MDS    RANDOM      TSNE      UMAP  
Dataset  Run                                            
auto-mpg Run 0  2.884478  4.986967  4.209013  4.552697  
         Run 1  2.877432  5.040332  4.230974  4.651954  
         Run 2  2.869343  5.013873  4.347048  4.518174  
         Run 3  2.873127  5.023361  4.224017  4.659922  
         Run 4  2.853428  5.008583  4.209997  4.597458  
         Run 5  2.868815  4.988555  4.207216  4.557358  
         Run 6  2.849332  5.031755  4.312349  4.487866  
         Run 7  2.873424  5.036935  4.193775  4.604635  
         Run 8  2.854037  5.015844  4.226895  4.618066  
         Run 9  2.863397  5.091042  4.216586  4.574389  
bank     Run 0  3.683140  7.270110  5.110395  2.905225  
         Run 1  3.596998  7.273468  5.083042  2.857095  
         Run 2  3.607411  7.274827  5.158812  2.873905  
         Run 3  3.730320  7.263903  5.174114  2.865848  
         Run 4  3.671364  7.281868  5.110306  2.871600

### KL Statistics For Each Algorithm

In [15]:
get_kl_stats(kl_at_scales.stack(level=0, future_stack=True).groupby('Scale'))

# Performance of Each Metric

### Percentages of Each Algorithm Ranking

In [16]:
def get_table(drop_UMAP: bool):
    data = {
        "Min-KL" : percentage_per_order(min_kls.loc[:, :, 'y'], drop_UMAP=drop_UMAP),
        "Normalized-KL": percentage_per_order(y_normalized_kls.loc[:, :, 'y'], drop_UMAP=drop_UMAP),
        "KL at Scale = 1": percentage_per_order(kl_at_scales[1], drop_UMAP=drop_UMAP),
        "KL at Scale = 5": percentage_per_order(kl_at_scales[5], drop_UMAP=drop_UMAP),
        "KL at Scale = 10": percentage_per_order(kl_at_scales[10], drop_UMAP=drop_UMAP),
        "KL at Scale = 20": percentage_per_order(kl_at_scales[20], drop_UMAP=drop_UMAP)
    }
    return pd.DataFrame(data).style.background_gradient('hot', vmin=0, vmax=100).format(lambda x : f"{x:.2f}%")


##### **Comparing t-SNE, MDS, Random**

In [17]:
get_table(drop_UMAP=True)

,Min-KL,Normalized-KL,KL at Scale = 1,KL at Scale = 5,KL at Scale = 10,KL at Scale = 20
TSNE < MDS < RANDOM,95.65%,80.00%,86.96%,60.00%,61.74%,47.39%
TSNE < RANDOM < MDS,0.00%,0.00%,0.00%,4.78%,3.91%,0.43%
MDS < TSNE < RANDOM,4.35%,20.00%,4.35%,5.22%,16.96%,24.78%
MDS < RANDOM < TSNE,0.00%,0.00%,8.70%,13.48%,13.04%,9.57%
RANDOM < TSNE < MDS,0.00%,0.00%,0.00%,0.87%,0.00%,0.00%
RANDOM < MDS < TSNE,0.00%,0.00%,0.00%,15.65%,4.35%,17.83%


##### **Comparing t-SNE, UMAP, MDS, Random**

In [18]:
get_table(drop_UMAP=False)

,Min-KL,Normalized-KL,KL at Scale = 1,KL at Scale = 5,KL at Scale = 10,KL at Scale = 20
TSNE < UMAP < MDS < RANDOM,82.17%,33.91%,82.17%,48.70%,22.17%,15.22%
TSNE < UMAP < RANDOM < MDS,0.00%,0.00%,0.00%,4.35%,3.91%,0.43%
TSNE < MDS < UMAP < RANDOM,12.17%,12.61%,0.43%,0.00%,0.00%,0.00%
TSNE < MDS < RANDOM < UMAP,0.00%,2.17%,0.00%,0.00%,0.00%,0.00%
TSNE < RANDOM < UMAP < MDS,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
TSNE < RANDOM < MDS < UMAP,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
UMAP < TSNE < MDS < RANDOM,1.30%,31.30%,4.35%,11.30%,39.57%,32.17%
UMAP < TSNE < RANDOM < MDS,0.00%,0.00%,0.00%,0.43%,0.00%,0.00%
UMAP < MDS < TSNE < RANDOM,0.00%,7.83%,0.00%,0.87%,8.26%,11.30%
UMAP < MDS < RANDOM < TSNE,0.00%,0.00%,3.04%,0.43%,0.00%,1.30%


### Correlation of Orders

In [19]:
def get_ranks(drop_UMAP: bool):
    ranks = {
        "Min-KL" : ranks_for_spearman(min_kls.loc[:, :, 'y'], drop_UMAP=drop_UMAP).values,
        "Y-Normalized-KL": ranks_for_spearman(y_normalized_kls.loc[:, :, 'y'], drop_UMAP=drop_UMAP).values,
        "KL at Scale = 1": ranks_for_spearman(kl_at_scales[1], drop_UMAP=drop_UMAP).values,
        "KL at Scale = 5": ranks_for_spearman(kl_at_scales[5], drop_UMAP=drop_UMAP).values,
        "KL at Scale = 10": ranks_for_spearman(kl_at_scales[10], drop_UMAP=drop_UMAP).values,
        "KL at Scale = 20": ranks_for_spearman(kl_at_scales[20], drop_UMAP=drop_UMAP).values,
    }

    correlation = pd.DataFrame(pd.NA, index=ranks.keys(), columns=ranks.keys()).astype("Float64")

    methods = list(ranks.keys())
    for i in range(len(methods)):
        for j in range(i, len(methods)):
            correlation.loc[methods[i], methods[j]] = spearmanr(ranks[methods[i]], ranks[methods[j]])[0]

    return correlation.style.set_caption("Without UMAP" if drop_UMAP else "With UMAP").background_gradient('Oranges_r', vmin=0, vmax=1).format(lambda x: '-' if pd.isna(x) else f'{x:.2f}')



#### **Considering only t-SNE, MDS, and Random**

In [20]:
get_ranks(drop_UMAP=True)


,Min-KL,Y-Normalized-KL,KL at Scale = 1,KL at Scale = 5,KL at Scale = 10,KL at Scale = 20
Min-KL,1.00,-0.11,0.58,0.22,0.33,0.11
Y-Normalized-KL,-,1.00,0.10,0.29,0.19,0.24
KL at Scale = 1,-,-,1.00,0.41,0.60,0.31
KL at Scale = 5,-,-,-,1.00,0.41,0.81
KL at Scale = 10,-,-,-,-,1.00,0.49
KL at Scale = 20,-,-,-,-,-,1.00


#### **Considering t-SNE, UMAP, MDS, Random**

* Orders are not meaningfully ranked, so correlation analysis is in question

In [21]:
get_ranks(drop_UMAP=False)

,Min-KL,Y-Normalized-KL,KL at Scale = 1,KL at Scale = 5,KL at Scale = 10,KL at Scale = 20
Min-KL,1.00,0.30,0.02,0.48,0.57,0.55
Y-Normalized-KL,-,1.00,0.28,0.15,0.32,0.33
KL at Scale = 1,-,-,1.00,0.14,0.14,0.15
KL at Scale = 5,-,-,-,1.00,0.47,0.60
KL at Scale = 10,-,-,-,-,1.00,0.81
KL at Scale = 20,-,-,-,-,-,1.00


#### **For Which Datasets is Expected Order Not Observed for Min-KL?**

In [22]:
def incorrect_datasets(df: pd.DataFrame, drop_UMAP=True):
    if drop_UMAP:
        order = ['TSNE', 'MDS', 'RANDOM']
    else:
        order = ['TSNE', 'UMAP', 'MDS', 'RANDOM']

    condition = pd.Series(False, index=df.index)
    for i in range(1, len(order)):
        condition |= df[order[i-1]] > df[order[i]]


    datasets = sorted(set(df[condition].index.get_level_values(level=0)))
    max_kl = df[condition].max(axis=None)
    not_preserved_kls = df[condition].style.background_gradient('inferno', vmax=max_kl)

    return datasets, not_preserved_kls

incorrect_datasets(min_kls.loc[:, :, 'y'])
bad_datasets, not_preserved_kls = incorrect_datasets(min_kls.loc[:, :, 'y'], drop_UMAP=True)
print(bad_datasets)
not_preserved_kls

['penguins']
